In [1]:
import json 
from datetime import datetime, timedelta
import re
import uuid

In [2]:
def reformat_time(milliseconds):
    seconds = milliseconds / 1000
    return str(timedelta(seconds=seconds))

In [3]:
def reformat_schedule(schedule_list):
    reformatted_schedule = []
    for schedule in schedule_list:
        days = [day.strip() for day in schedule['schedule'].split(',') if day]
        for day in days:
            reformatted_schedule.append({
                "day": day,
                "start_time": reformat_time(schedule['startTime']),
                "end_time": reformat_time(schedule['endTime'])
            })
    return reformatted_schedule

In [4]:
def generate_short_uuid():
    return str(uuid.uuid4())[:8]

In [5]:
def decode_unicode(text):
    return "Please see OneStop for course information."
    return text.encode('latin1').decode('unicode-escape')

In [9]:
def process_course(course):
    details_plain_text = decode_unicode(re.sub('<[^<]+?>', '', course['details']).replace('\n', ' '))
    
    formatted_course = {
        "title": course['name'],
        "id": course['courseCode'].replace(" ", "") + "_Fall2024_" + generate_short_uuid(),
        "credits": '4',
        "instructor": course['courseInstructor'],
        "url": "example.com",
        "description": details_plain_text,
        "categories": course['courseCategories'],
        "schedule": reformat_schedule(course['listCourseSchedule']) if course['listCourseSchedule'] else [],
        "location": course['listCourseSchedule'][0]['facility'] if course['listCourseSchedule'] else ""
    }
    
    return formatted_course

In [7]:
def reformat_courses(input_file, output_file):
    with open(input_file, 'r') as infile:
        courses = json.load(infile)
    
    formatted_courses = [process_course(course) for course in courses]
    
    with open(output_file, 'w') as outfile:
        json.dump(formatted_courses, outfile, indent=4, ensure_ascii=False)

In [10]:
file_in = "2024-2025_Spring_2025_Term_uncleaned.json"
file_out = "2024-2025 Spring 2025 Term.json"

reformat_courses(file_in, file_out)